# APP Testing - Analyzing Data Point File

Note: Search for "User edits here" to find where to edit the code


In [12]:
#Importing the libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob

plt.close('all')

In [13]:
#Read condition 1 data (Low suction pressure and high temp)

############################## User edits here ##############################

#Change the folder path where you store all condition 1 DP files
path = r'C:\Users\dtran\APP_Testing_DataAnalzying\data\cond1' 

############################## End editing ##############################

all_files = glob.glob(path + "/*.csv")

li = []


for filename in all_files:
    fn1_loc = filename.find("TestLog")
    fn2_loc = filename.find(".csv")
    fn = filename[fn1_loc:fn2_loc]
    df = pd.read_csv(filename, skiprows=50, index_col=None, header=0)
    df['FFT file name'] = fn + "_Raw Data Data Point " + df['Data Points'].astype(str) + ".tdms"
    df['FFT file path'] = path + "\\" + df['FFT file name'].astype(str)
    li.append(df)

data1 = pd.concat(li, axis=0, ignore_index=True)

data1['Condition'] = '1' #Critical condition
#print(data1)




In [14]:
#Read condition 2 data (normal suction pressure and temp)

############################## User edits here ##############################

#Change the folder path where you store all condition 2 DP files
path = r'C:\Users\dtran\APP_Testing_DataAnalzying\data\cond2'

############################## End editing ##############################

all_files = glob.glob(path + "/*.csv")

li = []


for filename in all_files:
    fn1_loc = filename.find("TestLog")
    fn2_loc = filename.find(".csv")
    fn = filename[fn1_loc:fn2_loc]
    df = pd.read_csv(filename, skiprows=50, index_col=None, header=0)
    df['FFT file name'] = fn + "_Raw Data Data Point " + df['Data Points'].astype(str) + ".tdms"
    df['FFT file path'] = path + "\\" + df['FFT file name'].astype(str)
    li.append(df)

data2 = pd.concat(li, axis=0, ignore_index=True)

data2['Condition'] = '2' #Normal condition
#print(data2)

In [15]:
if (data1.shape[1] != data2.shape[1]):
    print("Please check the data files. They do not have the same number of cols!")
else:
    print("Everything is alright!")
    #print(data1.shape[1])
    #print(data2.shape[1])



Everything is alright!


In [16]:
all_data = pd.concat([data1, data2])
#all_data = data2
#print(all_data)

In [17]:
#Correcting Volumetric and Hydro Mechanical efficiencies
Vdi = 0.095191784 #Calculated from the Q vs speed curve
all_data['Corrected Volumetric Efficiency']=all_data['Pump Flow']/(all_data['Pump Speed (from Torquemeter)']*Vdi)*100
all_data['Corrected Hydro Mechanical Efficiency']=all_data['Total Efficiency (from Torquemeter)']/all_data['Corrected Volumetric Efficiency']*100
all_data['DP']=(all_data['Pump Differential Pressure']/ 100).astype(int) *100
all_data['Speed']=(all_data['Pump Speed (from Torquemeter)']/ 100).astype(int) *100
#print(all_data)


In [18]:
#Filter the data based on the speed, e.g., 1500, 1300, 1100, 900, 700 rpm
speeds = [1500, 1300, 1100, 900, 700]

for speed in speeds:
    a = "is_speed_"
    b = str(speed)
    globals()[a + b] = all_data['VFD203_RPM_Control']==speed #is_speed_1500 format

    
#Filter the data based on DPs  
DP_threshold = 10
diff_pressures = [1100, 1000, 900, 800, 700, 600, 500, 450]
for diff_press in  diff_pressures:
    a1 = "is_DP_above_"
    a2 = "is_DP_below_"
    a3 = "is_DP_"
    b = str(diff_press)
    globals()[a1 + b] = all_data['Pump Differential Pressure']>(diff_press-DP_threshold)
    globals()[a2 + b] = all_data['Pump Differential Pressure']<(diff_press+DP_threshold)
    globals()[a3 + b] = globals()[a1 + b] & globals()[a2 + b]
    #print(globals()[a3 + b])

plt.close('all')
#data1 = data[(is_DP_450 | is_DP_500 | is_DP_600 | is_DP_700 | is_DP_800 | is_DP_900 | is_DP_1000 | is_DP_1100) & is_speed_700]
#data1 = data[(is_DP_450) & is_speed_700]
#print(data1.loc[:,"Total Efficiency (from Torquemeter)"].mean())
#print(data1.loc[:,"Total Efficiency (from Torquemeter)"].std())

#Plotting eff vs DP at certain speed
plot_speed = [700, 900, 1100, 1300, 1500]
plot_DP = [450, 500, 600, 700, 800, 900, 1000, 1100]


In [36]:
############################## User edits here ##############################

#Select condition to plot
plot_cond = '1'

############################## End editing ##############################


is_cond_right =  all_data['Condition']==plot_cond
#data = all_data[is_cond_right]
#print(data)

In [37]:
# plotting contour, ignore this step if you dont want to have a contour plot
plot_data = 'Corrected Hydro Mechanical Efficiency'
df=all_data[['Speed', 'DP', plot_data]]

#Averaging data
mean_Eff = []
speed_temp = []
DP_temp = []
    
for j in plot_DP:
    for i in plot_speed:
        df_temp = df[(globals()["is_DP_" + str(j)]) & (globals()["is_speed_" + str(i)]) & is_cond_right]
        #print(data_frame_temp)
        mean_Eff.append(df_temp.loc[:,plot_data].mean())
        speed_temp.append(i)
        DP_temp.append(j)

df_sorted = pd.DataFrame()        
df_sorted['Speed'] = speed_temp
df_sorted['DP'] = DP_temp
df_sorted[plot_data] = mean_Eff



df_sorted = df_sorted.pivot('Speed', 'DP', plot_data)
#print(df_sorted)
X=df_sorted.columns.values
#print(X)
Y=df_sorted.index.values
#print(Y)
Z=df_sorted.values
#print(Z)
x,y=np.meshgrid(X, Y)
#levels = [83, 84, 85, 86, 87, 88, 89, 90, 91, 92]
levels = np.linspace(82, 96, 50)
contour = plt.contour(x, y, Z, levels, colors='k')
plt.clabel(contour, inline=True, fontsize=15, fmt = '%2.1f')
contour_filled = plt.contourf(x, y, Z, levels)
plt.colorbar(contour_filled)
plt.title('Condition ' + plot_cond + ', ' + plot_data)
plt.xlabel('DP (psid)')
plt.ylabel('Speed (RPM)')
plt.show()

In [ ]:
############################## User edits here ##############################

#Select the parameter for Y axis for plotting
Y_axis_label = 'VT203'
#Set the upper and lower limit for the Y axis
Y_upper = 0.4
Y_lower = 0

############################## End editing ##############################


#Plotting a constant-speed curve

X_axis_label = 'Pump Differential Pressure'

for j in plot_speed:
    mean_Y = []
    std_Y = []
    mean_X = []
    std_X = []
    for i in plot_DP:
        #print((globals()["is_DP_" + str(i)]) & (globals()["is_speed_" + str(j)]))
        data_frame_temp = all_data[(globals()["is_DP_" + str(i)]) & (globals()["is_speed_" + str(j)]) & is_cond_right]
        #print(data_frame_temp)
        mean_Y.append(data_frame_temp.loc[:,Y_axis_label].mean())
        std_Y.append(data_frame_temp.loc[:,Y_axis_label].std())
        mean_X.append(data_frame_temp.loc[:,X_axis_label].mean())
        std_X.append(data_frame_temp.loc[:,X_axis_label].std())
    
    plt.errorbar(mean_X, mean_Y, xerr=std_X, yerr=std_Y, linestyle='-', marker='^', linewidth=2, fmt='--o', label="speed = " + str(j) + " rpm")
    
plt.xlabel(X_axis_label)
plt.ylabel(Y_axis_label)
#plt.title('Condition ' + plot_cond + ': ' + Y_axis_label + ' vs ' + X_axis_label)
plt.title('Condition ' + plot_cond)
#plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
plt.axis([400, 1200, Y_lower, Y_upper])
plt.legend(loc='best')
plt.grid(True)

plt.show()

#data1.plot(kind='scatter',x='Pump Differential Pressure',y='Total Efficiency (from Torquemeter)',color='red')
#plt.show()





#Plotting a constant-DP curve

X_axis_label = 'Pump Speed (from Torquemeter)'

for j in plot_DP:
    mean_Y = []
    std_Y = []
    mean_X = []
    std_X = []
    for i in plot_speed:
        data_frame_temp = all_data[(globals()["is_DP_" + str(j)]) & (globals()["is_speed_" + str(i)]) & is_cond_right]
        #print(data_frame_temp)
        mean_Y.append(data_frame_temp.loc[:,Y_axis_label].mean())
        std_Y.append(data_frame_temp.loc[:,Y_axis_label].std())
        mean_X.append(data_frame_temp.loc[:,X_axis_label].mean())
        std_X.append(data_frame_temp.loc[:,X_axis_label].std())
    
    plt.errorbar(mean_X, mean_Y, xerr=std_X, yerr=std_Y, linestyle='-', marker='^', linewidth=2, fmt='--o', label="DP = " + str(j) + " psid")
    
plt.xlabel(X_axis_label)
plt.ylabel(Y_axis_label)
#plt.title('Condition ' + plot_cond + ': ' + Y_axis_label + ' vs ' + X_axis_label)
plt.title('Condition ' + plot_cond)
#plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
plt.axis([600, 1600, Y_lower, Y_upper])
plt.legend(loc='best')
plt.grid(True)

plt.show()

#data1.plot(kind='scatter',x='Pump Differential Pressure',y='Total Efficiency (from Torquemeter)',color='red')
#plt.show()



In [ ]:
#Plot efficiency curves, e.g., overall, volumetric and hydro mechanical efficiencies, for a certain speed

############################## User edits here ##############################

#Select the parameter for X axis for plotting
X_axis_label = 'Pump Differential Pressure'
selected_speed = 1100

#Set the upper and lower limit for the Y axis
Y_upper = 100
Y_lower = 80

############################## End editing ##############################

Y_axis_parameters = ['Corrected Volumetric Efficiency', 'Corrected Hydro Mechanical Efficiency', 'Total Efficiency (from Torquemeter)']

for i in Y_axis_parameters:
    mean_Y = []
    std_Y = []
    mean_X = []
    std_X = []
    for j in plot_DP:
        data_frame_temp = all_data[(globals()["is_DP_" + str(j)]) & (globals()["is_speed_" + str(selected_speed)]) & is_cond_right]
        #print(data_frame_temp)
        mean_Y.append(data_frame_temp.loc[:,i].mean())
        std_Y.append(data_frame_temp.loc[:,i].std())
        mean_X.append(data_frame_temp.loc[:,X_axis_label].mean())
        std_X.append(data_frame_temp.loc[:,X_axis_label].std())
    
    plt.errorbar(mean_X, mean_Y, xerr=std_X, yerr=std_Y, linestyle='-', marker='^', linewidth=2, fmt='--o', label=i)   
    
    
plt.xlabel(X_axis_label)
plt.ylabel('Efficiency')
#plt.title('Condition ' + plot_cond + ': ' + Y_axis_label + ' vs ' + X_axis_label)
plt.title('Condition ' + plot_cond + ', Speed of ' + str(selected_speed) + ' rpm')
#plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
plt.axis([400, 1200, Y_lower, Y_upper])
plt.legend(loc='best')
plt.grid(True)

plt.show()

In [ ]:
#Plot FFT
from nptdms import TdmsFile
import scipy.fftpack

#FFT_plot_speed_array = [1500, 1300, 1100, 900, 700]
#FFT_plot_DP_array = [1100, 1000, 900, 800, 700, 600, 500, 450]
#FFT_plot_para_array = ['VT203D', 'PDYT201', 'PDYT202']
#FFT_plot_cond_array = ['1', '2']
FFT_plot_speed_array = [1100]
FFT_plot_DP_array = [1100]
FFT_plot_para_array = ['VT203D']
FFT_plot_cond_array = ['2']


for FFT_plot_speed in FFT_plot_speed_array:
    freq_1x = FFT_plot_speed/60.0
    for FFT_plot_DP in FFT_plot_DP_array:
        for FFT_plot_para in FFT_plot_para_array:
            if (FFT_plot_para == 'VT203D') | (FFT_plot_para == 'VT203AD'):
                coeff = 10
            else:
                coeff = 1
                
            for FFT_plot_cond in FFT_plot_cond_array:
                
                is_FFT_cond_right =  all_data['Condition']==FFT_plot_cond

                #Filter out speed, DP, condition you want to plot
                data_frame_temp = all_data[(globals()["is_DP_" + str(FFT_plot_DP)]) & (globals()["is_speed_" + str(FFT_plot_speed)]) & is_FFT_cond_right]
                #print(data_frame_temp)
                count = 0

                for i in data_frame_temp['FFT file path']:
                    count = count + 1
                    tdms_file = TdmsFile(i)
                    MessageData_channel_1 = tdms_file.object('Fast Channels Raw Data', FFT_plot_para)
                    MessageData_data_1 = MessageData_channel_1.data

                    Fs = 6000.0
                    deltaT = 1/Fs 
                    L = len(MessageData_data_1)
                    t = np.arange(0, L-1, 1)*deltaT

                    maxTime = deltaT*(L-1);
                    time = np.linspace(0,maxTime,L)
                    disp = MessageData_data_1

                    f = np.arange(0, L/2, 1)*Fs/L
                    f_x = f/freq_1x
                    X_fft = scipy.fftpack.fft(disp)
                    P2_X = coeff*abs(X_fft/L)
                    P1_X = P2_X[1:L/2+1]
                    end = len(P1_X)
                    P1_X[2:end-1] = 2*P1_X[2:end-1]

                    plt.xlabel('f (1x)')
                    plt.ylabel(FFT_plot_para)
                    plt.title(FFT_plot_para + ': Condition ' + FFT_plot_cond + ', ' + 'Speed of ' + str(FFT_plot_speed) + ' rpm, ' + 'DP of ' + str(FFT_plot_DP) + ' psid, Point ' + str(count))
                    plt.plot(f_x, P1_X, color='green', marker='o', linestyle='dashed', linewidth=2, markersize=3)
                    #plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
                    plt.xlim(0, 60)
                    plt.ylim(0, 0.2)
                    plt.xticks(np.arange(0, 60 , 9))
                    #plt.axis.set_xlim(400, 1200)
                    #plt.legend(loc='best')
                    plt.grid(True)
                    file_name = FFT_plot_para + ', Condition ' + FFT_plot_cond + ', ' + 'Speed of ' + str(FFT_plot_speed) + ' rpm, ' + 'DP of ' + str(FFT_plot_DP) + ' psid, Point ' + str(count) + '.png'
                    file_path = 'C:\\Users\\dtran\\APP_Testing_DataAnalzying\\data\\axialvibration0507\\saved_FFT\\' + file_name
                    #plt.savefig(file_path)

                    plt.show()
                    plt.close()
                    #break